In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class VanillaNet(nn.Module):
    def __init__(self, n_c=3, n_f=32, leak=0.05):
        super(VanillaNet, self).__init__()
        self.f = nn.Sequential(
            nn.Conv2d(n_c, n_f, 3, 1, 0),
            nn.LeakyReLU(leak),
            nn.Conv2d(n_f, n_f * 2, 4, 2, 0),
            nn.LeakyReLU(leak),
            nn.Conv2d(n_f*2, n_f*4, 4, 2, 0),
            nn.LeakyReLU(leak),
            nn.Conv2d(n_f*4, n_f*8, 4, 2, 0),
            nn.LeakyReLU(leak),
            nn.Conv2d(n_f*8, 1, 4, 1, 0))

    def forward(self, x):
        return self.f(x).squeeze()



net = VanillaNet()
print(net)

VanillaNet(
  (f): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.05)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): LeakyReLU(negative_slope=0.05)
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
    (5): LeakyReLU(negative_slope=0.05)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
    (7): LeakyReLU(negative_slope=0.05)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
  )
)


In [3]:
trained_model = VanillaNet()

In [4]:
trained_model.load_state_dict(torch.load('model/anila_flowers/saved_model.pth',map_location='cpu'))

<All keys matched successfully>

In [5]:
print(trained_model)

VanillaNet(
  (f): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.05)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): LeakyReLU(negative_slope=0.05)
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
    (5): LeakyReLU(negative_slope=0.05)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
    (7): LeakyReLU(negative_slope=0.05)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
  )
)


In [115]:
from torch.autograd import Variable
# Export the trained model to ONNX
dummy_input = Variable(torch.randn(32, 3, 128, 128))
torch.onnx.export(trained_model, dummy_input, "model/anila_flowers/flowers.onnx")

In [6]:
import onnx

ImportError: No module named onnx

In [120]:
#Load the ONNX file
model = onnx.load('model/anila_flowers/flowers.onnx')

NameError: name 'onnx' is not defined

In [ ]:
from onnx_tf.backend import prepare
# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

In [ ]:
#Save the Tensorflow model into a file
tf_rep.export_graph('model/anila_flowers/flowers.pb')

## Stitch images

In [1]:
import sys
from PIL import Image

In [12]:
path = './saved/results/anila/'
images = [Image.open(x) for x in [path+'0000.jpg',
                                 path+'0001.jpg',
                                 path+'0002.jpg',
                                 path+'0003.jpg',
                                 path+'0004.jpg']]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for im in images:
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]

new_im.save('./saved/results/anila/merged.jpg')